$$
\frac{\partial P_i}{\partial t} = \mu_{ref} \cdot T_f \cdot \left(1- exp\left(\frac{-\alpha_i^{Chl}\ \theta_i^C\ I}{\mu_{ref}\ T_f\ V_i}\right)\right) \cdot V_i \cdot P_i - g_z^{\max} \cdot Z \cdot \frac{P_i}{K_{P_i} + P_i} - m_i \cdot T_f \cdot P_i - \alpha_i \cdot P_i^{1.75}  $$

$\mu_{ref} = 5.0 d^{-1}$                                   Maximum C-spec growth rate at $T_{ref}$ for small phytoplankton and diatoms  
$T_{ref} = 30 ^\circ C$                                    Refrence temperature  
$\theta_{max,sp}^N = 2.5\ mg Chl/mmol$                     Maximum $\theta^N (Chl/N)$ for small phytoplankton  
$\theta_{max,diat}^N = 4.0\ mg Chl/mmol$                   Maximum $\theta^N (Chl/N)$ for diatoms  
$\alpha_{sp}^{Chl} = 0.39\ mmol\ m^2/(mg\ Chl\ W\ day)$    Chl-spec initial slope of P_I curve for small phytoplankton
$\alpha_{diat}^{Chl} = 0.28\ mmol\ m^2/(mg\ Chl\ W\ day)$  Chl-spec initial slope of P_I curve for diatoms  
$K_{sp}^{Dfe} = 3.0e-05\ mmol/m^3$                         Fe uptake half-sat constant for small phytoplankton  
$K_{diat}^{Dfe} = 7.0e-05\ mmol/m^3$                       Fe uptake half-sat constant for diatoms  
$K_{sp}^{NO^3} = 0.25\ mmol/m^3$                           $NO^3$ uptake half-sat constant for small phytoplankton

The maximum C-spec growth rate at \( T_{ref} \) for small phytoplankton and diatoms is given by:

$\mu_{ref} = 5.0\; d^{-1}$  

The reference temperature is given by \( T_{ref} = 30^\circ\; \mathrm{C} \).

For small phytoplankton:

$\theta_{max,sp}^N = 2.5\; mg\; Chl/mmol$  

For diatoms:

$\theta_{max,diat}^N = 4.0\; mg\; Chl/mmol$  

The Chl-spec initial slope of the P-I curve for small phytoplankton is:

$\alpha_{sp}^{Chl} = 0.39\; \mathrm{mmol}\; m^{-2}\; (mg\; Chl\; W\; day)^{-1}$  

And for diatoms:

$\alpha_{diat}^{Chl} = 0.28\; \mathrm{mmol}\; m^{-2}\; (mg\; Chl\; W\; day)^{-1}$

In [1]:
import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchdiffeq import odeint_adjoint as odeint
import pandas as pd
import numpy as np
import xarray as xr
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader

In [2]:
# Neuromancer imports (symbolic API)
import neuromancer as nm
from neuromancer.problem import Problem
from neuromancer.loss import PenaltyLoss
from neuromancer.trainer import Trainer
from neuromancer.system import Node, System
from neuromancer.dynamics import integrators, ode
from neuromancer.dataset import DictDataset
from neuromancer.constraint import variable

In [4]:
ds = xr.open_dataset("regridded_data/2015-2021_combined.nc")

In [5]:
mu_ref = 5.0
k_dfe = 3 ** (-0.5)
k_no3 = 0.5
k_po4 = 0.01
ds['V_dfe'] = ds['dfe'] / (k_dfe + ds['dfe'])
ds['V_no3'] = ds['no3'] / (k_no3 + ds['no3'])
ds['V_po4'] = ds['po4'] / (k_po4 + ds['po4'])
ds['Tf'] = 1.7 ** np.exp((ds['sst'] - 30) / 10)

# Convert to NumPy arrays, compute the minimum, and then convert back to xarray
V_no3_array = ds['V_no3'].values  # Convert to NumPy array
V_po4_array = ds['V_po4'].values  # Convert to NumPy array
V_dfe_array = ds['V_dfe'].values   # Assuming V_dfe is already defined

# Calculate the minimum using NumPy
V_min = np.minimum(np.minimum(V_dfe_array, V_po4_array), V_no3_array)

# Assign the result back to the DataArray in the xarray Dataset
ds['V'] = xr.DataArray(V_min, dims=ds['V_dfe'].dims, coords=ds['V_dfe'].coords)

df = ds.to_dataframe()
df = df.dropna(how='any')
scaler = MinMaxScaler(feature_range=(0,10))
scaled_array = scaler.fit_transform(df)
norm_df = pd.DataFrame(scaled_array, index=df.index, columns=df.columns)
print(norm_df.head())
print(norm_df.info())


                              dfe       no3       po4       sil       rsn  \
lon    lat   month_year                                                     
-179.5 -76.5 2015-01-01  0.042463  2.457866  5.414602  2.663714  6.223525   
             2015-02-01  0.056386  2.517316  5.486631  2.416398  3.783490   
             2015-12-01  0.048093  2.668628  5.709538  3.263838  8.403666   
             2016-01-01  0.033411  2.665089  5.803986  3.021275  5.299753   
             2016-02-01  0.066006  2.360248  5.209874  2.289098  3.853204   

                              mld       ssh       sal       sst      chla  \
lon    lat   month_year                                                     
-179.5 -76.5 2015-01-01  0.031551  1.773168  7.995484  0.429358  0.053512   
             2015-02-01  0.096969  1.718363  8.200290  0.225703  0.076863   
             2015-12-01  0.076738  1.647681  8.305805  0.355078  0.401371   
             2016-01-01  0.089604  1.587883  8.291054  0.466634  0.112831  

In [6]:
# Resetting the index to work with columns directly
df = norm_df.sort_index().reset_index()
df['month_year'] = pd.to_datetime(df['month_year'])
start_date = df['month_year'].min()
# df['months_from_start'] = (df['month_year'] - df['month_year'].min()) // pd.Timedelta('30D')
df['months_from_start'] = ((df['month_year'].dt.year - start_date.year) * 12 + 
                                  (df['month_year'].dt.month - start_date.month))
# Count months per (lon, lat)
month_counts = df.groupby(['lon', 'lat'])['months_from_start'].nunique()
expected_months = df['months_from_start'].nunique()
valid_locations = month_counts[month_counts == expected_months].index

# Filter the DataFrame
df_filtered = df.set_index(['lon', 'lat']).loc[valid_locations].reset_index()
df_filtered = df_filtered.sort_values(['lon', 'lat', 'months_from_start'])
print(df_filtered.shape)
df_sampled = df_filtered.sample(n=1000, random_state=42).sort_values(['lon', 'lat', 'months_from_start'])
print(df_sampled.shape)

(66744, 26)
(1000, 26)


In [7]:
# Define input and target columns
input_cols = ['rsn', 'Tf', 'V', 'logchla']
target_col = ['logmcro']

# Group by location
grouped = df_sampled.groupby(['lon', 'lat'])

X_list, Y_list = [], []
for _, group in grouped:
    group = group.sort_values('months_from_start')
    X = group[input_cols].values
    Y = group[target_col].values
    X_list.append(torch.tensor(X, dtype=torch.float32))
    Y_list.append(torch.tensor(Y, dtype=torch.float32))

In [8]:
# needed only if the sequences are of different lengths
nsteps = max(len(x) for x in X_list)
X_padded = torch.stack([torch.nn.functional.pad(x, (0, 0, 0, nsteps - x.shape[0])) for x in X_list])
Y_padded = torch.stack([torch.nn.functional.pad(y, (0, 0, 0, nsteps - y.shape[0])) for y in Y_list])

X_padded = X_padded.view(len(X_list), nsteps, -1)  # shape: (nbatch, nsteps, n_features)
Y_padded = Y_padded.view(len(Y_list), nsteps, -1)  # shape: (nbatch, nsteps, 1)

In [9]:
# Set split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Compute split indices
nbatch = len(X_list)
n_train = int(train_ratio * nbatch)
n_val = int(val_ratio * nbatch)
n_test = nbatch - n_train - n_val

# Shuffle indices for reproducibility
torch.manual_seed(42)
indices = torch.randperm(nbatch)

train_idx = indices[:n_train]
val_idx = indices[n_train:n_train + n_val]
test_idx = indices[n_train + n_val:]

X_train = X_padded[train_idx]
Y_train = Y_padded[train_idx]

X_val = X_padded[val_idx]
Y_val = Y_padded[val_idx]

X_test = X_padded[test_idx]
Y_test = Y_padded[test_idx]

train_data = DictDataset({
    'Y0':Y_train[:, 0, :],   # initial Pi (shape: [B, 1])
    'X': X_train,            # external inputs (shape: [B, T, 4])
    'Y': Y_train             # ground truth Pi sequence
}, name='train')

val_data = DictDataset({
    'Y0': Y_val[:, 0, :],
    'X': X_val,
    'Y': Y_val
}, name='val')

test_data = DictDataset({
    'Y0': Y_test[:, 0, :],
    'X': X_test,
    'Y': Y_test
}, name='test')

train_loader = DataLoader(train_data, batch_size=32, collate_fn=train_data.collate_fn, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, collate_fn=val_data.collate_fn, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, collate_fn=test_data.collate_fn, shuffle=False)

In [10]:
from neuromancer import blocks

# define UDE system in Neuromancer
class PlanktonHybrid(ode.ODESystem):
    def __init__(self, block, insize=5, outsize=1):
        """
        :param block: Neural block (e.g. MLP) that learns unknown dynamics
        :param insize: Number of input features (rsn, Tf, V, logchla)
        :param outsize: Number of output features (logmcro)
        """
        super().__init__(insize=insize, outsize=outsize)
        self.block = block
        
        # Mechanistic parameters
        self.mu_ref = torch.tensor(5.0)
        self.alpha_chl =torch.tensor(0.28)
        self.m_i = nn.Parameter(torch.tensor(0.05), requires_grad=True)
        self.alpha_i = nn.Parameter(torch.tensor(0.01), requires_grad=True)
        assert self.block.in_features == 5
        assert self.block.out_features == 2
        
    def ode_equations(self, Pi, U):
        """
        Pi: shape (B, 1) — current plankton biomass
        U: shape (B, 4) — external inputs [I, Tf, V, logchla]
        """
        # Concatenate inputs and state
        xu = torch.cat([U, Pi], dim=-1)  # shape: (B, 5)

        # Neural block outputs
        learned_theta, learned_graz = torch.split(self.block(xu), 1, dim=-1)
        # ensure positive grazing via softplus (or relu)
        learned_graz = F.softplus(learned_graz_raw)

        # Extract environmental drivers
        I  = U[:, [0]]
        Tf = U[:, [1]]
        V  = U[:, [2]]
        logchla = U[:, [3]]    

        # Mechanistic dynamics
        denom = torch.clamp(self.mu_ref * Tf * V, min=1e-6)
        exponent = torch.clamp(-self.alpha_chl * learned_theta * I / denom, min=-50.0, max=50.0)
        growth = self.mu_ref * Tf * (1 - torch.exp(exponent)) * V * Pi
        mortality = self.m_i * Tf * Pi
        aggregation = self.alpha_i * (Pi ** 1.75)

        # Final dynamics
        dPi = growth - learned_graz - mortality - aggregation
        
        # final safety clamp
        dPi = torch.nan_to_num(dPi, nan=0.0, posinf=1e6, neginf=-1e6)
        return dPi

# Define a neural block with 5 inputs and 2 outputs
neural_block = blocks.MLP(
    insize=5,
    outsize=2,
    hsizes=[64, 64],             # two hidden layers
    nonlin=torch.nn.ReLU,        # activation function
    bias=True,
    linear_map=torch.nn.Linear
)

# Non-autonomous
class PlanktonNode(Node):
    def forward(self, Y0, X):
        """
        Y0: initial state, shape (B, state_dim)
        X: sequence of inputs, shape (B, T, input_dim)
        returns: Y_pred of shape (B, T, state_dim)
        """
        B, T, _ = X.shape
        Pi = Y0
        Y_pred_seq = []

        for t in range(T):
            U_t = X[:, t, :]       # shape (B, input_dim)
            Pi = fxRK4(Pi, U_t)   # RK4 step: Pi (B, state_dim), U_t (B, input_dim)
            Y_pred_seq.append(Pi)

        Y_pred = torch.stack(Y_pred_seq, dim=1)  # (B, T, state_dim)
        return Y_pred


In [13]:
# Instantiate your hybrid system
fx = PlanktonHybrid(block=neural_block)

ts = 1.0  # time step size (e.g. 1 month)
# integrate UDE model
fxRK4 = integrators.RK4(fx, h=ts)

# create symbolic UDE model
plankton_node = Node(
    callable=fxRK4,
    input_keys=['Y0', 'X'],      # Y0 = initial Pi, X = external inputs
    output_keys=['Y_pred'],     # predicted output sequence
    name='PlanktonUDE'
)
dynamics_model = System(
    nodes=[plankton_node],
    nsteps=nsteps             # e.g. 72 months
)

# define symbolic losses
Y = variable('Y')             # true sequence
Y_pred = variable('Y_pred')   # predicted sequence

# trajectory tracking loss
tracking_loss = 1.0 * ((Y_pred == Y)^2)
tracking_loss.name = 'tracking_loss'

# One-step loss
onestep_loss = 1.0 * ((Y_pred[:, 1, :] == Y[:, 1, :])^2)
onestep_loss.name = 'onestep_loss'

# Finite difference loss
Y_fd = Y[:, 1:, :] - Y[:, :-1, :]
Y_pred_fd = Y_pred[:, 1:, :] - Y_pred[:, :-1, :]
fd_loss = 2.0 * ((Y_fd == Y_pred_fd)^2)
fd_loss.name = 'FD_loss'

# Initial condition consistency
init_loss = (Y_pred[:, 0, :] == Y[:, 0, :])^2
init_loss.name = 'init_loss'

# # parameter regularisation loss
# alpha_i = variable('PlanktonUDE.alpha_i')
# m_i = variable('PlanktonUDE.m_i')
# reg_loss = 0.01 * (alpha_i ** 2 + m_i ** 2)
# reg_loss.name = 'symbolic_reg'

# add weights
tracking_loss.weight = 1.0
onestep_loss.weight = 1.0
fd_loss.weight = 2.0
init_loss.weight = 0.5
# reg_loss.weight = 0.01

# aggregate list of objective terms and constraints
objectives = [tracking_loss, onestep_loss, fd_loss, init_loss]
constraints = []

# create constrained optimization loss
loss = PenaltyLoss(objectives, constraints)
# construct constrained optimization problem
problem = Problem(
    [dynamics_model], 
    loss,
    grad_inference=True     # enables symbolic gradient flow
)

In [14]:
import csv, os

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

# device = 'cuda' if torch.cuda.is_available() elif device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)
problem.to(device)
optimizer = torch.optim.Adam(problem.parameters(), lr=0.003)

trainer = Trainer(
    problem,
    train_loader,
    val_loader,
    test_data,
    optimizer,
    patience=5,
    warmup=5,
    epochs=10,
    eval_metric="val_loss",
    train_metric="train_loss",
    dev_metric="val_loss",
    test_metric="val_loss",
)

cuda


In [15]:
best_model = trainer.train()
problem.load_state_dict(best_model)

AssertionError: 